# Libraries and PySpark Session

In [2]:
pip install spark-nlp==3.3.4

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install numpy pandas nltk

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
from pyspark.sql import SparkSession

In [2]:
SparkSession.builder.appName("BDA_Reddit_Preprocessing").getOrCreate()

In [3]:
from sparknlp.annotator import LemmatizerModel
import pyspark.sql.functions as f
from pyspark.sql import Window
import pyspark.sql.types as t
from pyspark.ml.feature import Tokenizer as pysparkTokenizer, HashingTF, StopWordsRemover, CountVectorizer
from pyspark.ml import Pipeline
from sparknlp.base import DocumentAssembler, Finisher
from sparknlp.annotator import Tokenizer, Normalizer, LemmatizerModel, StopWordsCleaner, PerceptronModel, Chunker
from pyspark.ml.feature import StopWordsRemover, CountVectorizer, IDF
from pyspark.ml.clustering import LDA
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

/opt/conda/miniconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/bda_reddit_pw/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Tests

In [4]:
# Whole data or just a single batch?

df = spark.read.json("stream_reddit/97410eb3-*.json").distinct()
# df = spark.read.json("stream_reddit/*.json").distinct()

In [11]:
df.printSchema()

root
 |-- comments: long (nullable = true)
 |-- id: string (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- text: string (nullable = true)
 |-- time: double (nullable = true)
 |-- title: string (nullable = true)
 |-- upvotes: long (nullable = true)



In [12]:
df = df.select("id", "subreddit", "title", "text", "time")

Concatenating text

In [13]:
df = df.select("id", "time", "subreddit", f.concat_ws(" ", df.title, df.text).alias("text"))

In [14]:
df.count()

2538

## NLP-related tasks

In [11]:
from sparknlp.annotator import Lemmatizer, Stemmer, Tokenizer, Normalizer
from sparknlp.base import DocumentAssembler, Finisher

In [12]:
# Spark Pipelines allow us to sequentially add components such as transformers
from pyspark.ml import Pipeline

# These are components we will incorporate into our pipeline.
from pyspark.ml.feature import StopWordsRemover, CountVectorizer, IDF

In [13]:
from nltk.corpus import stopwords
eng_stopwords = stopwords.words('english')

In [14]:
# # https://towardsdatascience.com/natural-language-processing-with-pyspark-and-spark-nlp-b5b29f8faba

# documentAssembler = DocumentAssembler() \
#      .setInputCol('text') \
#      .setOutputCol('document')
# tokenizer = Tokenizer() \
#      .setInputCols(['document']) \
#      .setOutputCol('token')
# # note normalizer defaults to changing all words to lowercase.
# # Use .setLowercase(False) to maintain input case.
# normalizer = Normalizer() \
#      .setInputCols(['token']) \
#      .setOutputCol('normalized') \
#      .setLowercase(True)
# # note that lemmatizer needs a dictionary. So I used the pre-trained
# # model (note that it defaults to english)
# lemmatizer = LemmatizerModel.pretrained() \
#      .setInputCols(['normalized']) \
#      .setOutputCol('lemma')
# stopwords_cleaner = StopWordsCleaner() \
#      .setInputCols(['lemma']) \
#      .setOutputCol('clean_lemma') \
#      .setCaseSensitive(False) \
#      .setStopWords(eng_stopwords)
# # finisher converts tokens to human-readable output
# finisher = Finisher() \
#      .setInputCols(['clean_lemma']) \
#      .setCleanAnnotations(False)

In [15]:
# pipeline = Pipeline() \
#      .setStages([
#            documentAssembler,
#            tokenizer,
#            normalizer,
#            lemmatizer,
#            stopwords_cleaner,
#            finisher
#      ])

In [16]:
# # https://codelabs.developers.google.com/codelabs/spark-nlp#7

# Now, we begin assembling our pipeline. Each component here is used to some transformation to the data.
# The Document Assembler takes the raw text data and convert it into a format that can
# be tokenized. It becomes one of spark-nlp native object types, the "Document".
document_assembler = DocumentAssembler().setInputCol("text").setOutputCol("document")

# The Tokenizer takes data that is of the "Document" type and tokenizes it.
# While slightly more involved than this, this is effectively taking a string and splitting
# it along ths spaces, so each word is its own string. The data then becomes the
# spark-nlp native type "Token".
tokenizer = Tokenizer().setInputCols(["document"]).setOutputCol("token")

# The Normalizer will group words together based on similar semantic meaning.
normalizer = Normalizer().setInputCols(["token"]).setOutputCol("normalizer")

# The Stemmer takes objects of class "Token" and converts the words into their
# root meaning. For instance, the words "cars", "cars'" and "car's" would all be replaced
# with the word "car".
stemmer = Stemmer().setInputCols(["normalizer"]).setOutputCol("stem")

# The Finisher signals to spark-nlp allows us to access the data outside of spark-nlp
# components. For instance, we can now feed the data into components from Spark MLlib.
finisher = Finisher().setInputCols(["stem"]).setOutputCols(["to_spark"]).setValueSplitSymbol(" ")

# Stopwords are common words that generally don't add much detail to the meaning
# of a body of text. In English, these are mostly "articles" such as the words "the"
# and "of".
stopword_remover = StopWordsRemover(inputCol="to_spark", outputCol="filtered")

# Here we implement TF-IDF as an input to our LDA model. CountVectorizer (TF) keeps track
# of the vocabulary that's being created so we can map our topics back to their
# corresponding words.
# TF (term frequency) creates a matrix that counts how many times each word in the
# vocabulary appears in each body of text. This then gives each word a weight based
# on its frequency.
tf = CountVectorizer(inputCol="filtered", outputCol="raw_features")

# Here we implement the IDF portion. IDF (Inverse document frequency) reduces
# the weights of commonly-appearing words.
idf = IDF(inputCol="raw_features", outputCol="features")

In [17]:
lda = LDA(k=10, maxIter=10)

# We add all of the transformers into a Pipeline object. Each transformer
# will execute in the ordered provided to the "stages" parameter
pipeline = Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        normalizer,
        stemmer,
        finisher,
        stopword_remover,
        tf,
        idf,
        lda
    ]
)

In [18]:
text = df.select("text").fillna("")

In [19]:
model = pipeline.fit(text)

22/12/30 16:12:17 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 3.0 in stage 11.0 (TID 217) (bda-hdfs-dataproc-new-w-1.europe-north1-b.c.crypto-busting.internal executor 1): java.lang.ClassCastException: cannot assign instance of java.lang.invoke.SerializedLambda to field org.apache.spark.sql.catalyst.expressions.objects.LambdaVariable.accessor of type scala.Function2 in instance of org.apache.spark.sql.catalyst.expressions.objects.LambdaVariable
	at java.io.ObjectStreamClass$FieldReflector.setObjFieldValues(ObjectStreamClass.java:2301)
	at java.io.ObjectStreamClass.setObjFieldValues(ObjectStreamClass.java:1431)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2437)
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2355)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2213)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1669)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.ja

Py4JJavaError: An error occurred while calling o107.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 11.0 failed 4 times, most recent failure: Lost task 3.3 in stage 11.0 (TID 224) (bda-hdfs-dataproc-new-w-0.europe-north1-b.c.crypto-busting.internal executor 3): java.lang.ClassCastException: cannot assign instance of java.lang.invoke.SerializedLambda to field org.apache.spark.sql.catalyst.expressions.objects.LambdaVariable.accessor of type scala.Function2 in instance of org.apache.spark.sql.catalyst.expressions.objects.LambdaVariable
	at java.io.ObjectStreamClass$FieldReflector.setObjFieldValues(ObjectStreamClass.java:2301)
	at java.io.ObjectStreamClass.setObjFieldValues(ObjectStreamClass.java:1431)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2437)
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2355)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2213)
	at java.io.ObjectInp

In [ ]:
from pyspark.sql.functions import explode, col
words = text.withColumn('exploded_text', 
                               explode(col('clean_lemma')))

In [ ]:
counts = words.groupby('exploded_text').count()
counts_pd = counts.toPandas()
equifax_dict = {counts_pd.loc[i, 'exploded_text']: 
                counts_pd.loc[i, 'count'] 
                for i in range(counts_pd.shape[0])}

## crypto stream

In [4]:
df = spark.read.json("stream_crypto/*.json").distinct()

ERROR:root:Exception while sending command.                  (1860 + 4) / 10000]
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1211, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while receiving


Py4JError: An error occurred while calling o73.json